# Enigma: functional programming project

In [1]:
# Encryption Rotor function
def rotor(symbol, n, reverse=False):
    alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    rotors = {0: 'ABCDEFGHIJKLMNOPQRSTUVWXYZ',
          1: 'EKMFLGDQVZNTOWYHXUSPAIBRCJ',
          2: 'AJDKSIRUXBLHWTMCQGZNPYFVOE',
          3: 'BDFHJLCPRTXVZNYEIWGAKMUSQO'
          }
    disc = rotors.get(n)
    if not reverse:
        index = alphabet.index(symbol)
        encrypted = disc[index]
        return encrypted
    else:
        index = disc.index(symbol)
        decrypted = alphabet[index]
        return decrypted

In [2]:
# Encryprion Reflector function
def reflector(symbol, n):   
    reflectors = {0: 'ABCDEFGHIJKLMNOPQRSTUVWXYZ',
                  1: 'YRUHQSLDPXNGOKMIEBFZCWVJAT',
                  2: 'FVPJIAOYEDRZXWGCTKUQSBNMHL',
                  3: 'ENKQAUYWJICOPBLMDXZVFTHRGS',
                  4: 'RDOBJNTKVEHMLFCWZAXGYIPSUQ',
                  }
    A, B = ((reflectors[n], reflectors[0]))
    return A[B.index(symbol)]

We have N rotors of different types and a reflector.

The signal passes through the rotors sequentially. That is, inside the rotor, one symbol is replaced by another according to the rotor specification (we have implemented 4 such symbols).

Then the signal is transmitted from one Rotor to another. In this case, the Rotors can be either combined (i.e., opposite each output symbol of the 1 rotor is the same symbol of the 2 rotor), or shifted (in fact, this is Caesar encryption with some kind of shift).

The signal reaches the reflector, which is also encrypted.

The signal begins the reverse journey through the rotors, being encrypted on each of them (with a reverse offset), and also, if the rotors are not combined, then it is encrypted with the Caesar cipher when switching from one Rotor to another.

<img src='Enigma.png'/>

More about how Enigma works: https://www.codesandciphers.org.uk/enigma/example1.htm

# Enigma v.0.1

Let's create a simplified version of Enigma with the help of rotor and reflector functions above. In this version we will use 3 rotors and 1 reflector. There will be no displacements and no movement of the rotors/reflectors.

We need to implement the function enigma(text, ref, rot1, rot2, rot3), where:

    text - the source text to be encrypted
    ref - the number of the reflector
    rot1, rot2, rot3 - the numbers of the rotors

In [3]:
def enigma(text, ref, rot1, rot2, rot3):
    encrypted = ''
    text = [i for i in text if i.isalpha()]
    text = ''.join(text).upper()
    for letter in text:
        symbol1 = rotor(letter, rot3, reverse=False)
        symbol2 = rotor(symbol1, rot2, reverse=False)
        symbol3 = rotor(symbol2, rot1, reverse=False)
        symbol4 = reflector(symbol3, ref)
        symbol5 = rotor(symbol4, rot1, reverse=True)
        symbol6 = rotor(symbol5, rot2, reverse=True)
        symbol7 = rotor(symbol6, rot3, reverse=True)
        encrypted += symbol7
    return encrypted

We already know that rotors are used for encryption in Enigma. But they can also be rotated around their axis. In Enigma, it is not the displacement of the rotors relative to each other that is set, but the position of the rotors relative to the axis.

In fact, it means that Caesar cipher (about this technique you can read in next cell) is used not only when the signal passes from the rotor to the rotor, but also BEFORE the signal from the keyboard hits the rightmost rotor, as well as AFTER passing the right rotor (before exiting).

We will set the position of each rotor by a number (from 0 to 25).

Do not forget that a positive offset on the road "there" turns into a negative offset on the road "back".

Key points:

    1) The offset when hitting the right rotor from the keyboard is equal to the displacement of the rotor relative to the axis
    2) The displacement when hitting from one rotor to another is equal to the difference between the displacement of the rotors relative to the axis
    3) The displacement when hit from the left rotor to the reflector is equal to the displacement of this rotor relative to the axis and back in sign
    4) The displacement when hit from the right rotor to the indicator (to the output) is equal to the displacement of the rotor relative to the axis and back along the sign

**Caesar cipher**

To take the rotation of the rotors into account, it is necessary to create caesar function, which will allow us to use Caesar cipher encryprion technique. Here we replace each character with another according to a certain rule, "shifting" it in the alphabet. This is what we do for each character.

<img src='caesar_cipher.png'/>

The Caesar cipher with a shift of 3:

    A is replaced by D
    B is replaced by E
    and so on
    Z is replaced by C

So if we reach the end of the alphabet, then we start again.

# Enigma v.0.2

We will implement the function enigma(text, ref, rot1, shift1, rot2, shift2, rot3, shift3) with rotating motors, as they are described in the previous step.

    text - the source text to be encrypted
    ref - the number of the reflector
    rot1, rot2, rot3 - the numbers of the rotors
    shift1, shift2, shift3 - the shifts of the rotors (1, 2 and 3 respectively)

In [4]:
# Caesar cipher function
def caesar(text, key, alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'):
    text = [i for i in text if i.isalpha() or i.isdigit()]
    text = ''.join(text).upper()
    encrypted = ''
    for letter in text:
        index = (alphabet.index(letter) + key) % (len(alphabet))
        encrypted += alphabet[index]
    return encrypted

In [5]:
# Enigma v.0.2 function
def enigma(text, ref, rot1, shift1, rot2, shift2, rot3, shift3):
    encrypted = ''
    text = [i for i in text if i.isalpha()]
    text = ''.join(text).upper()
    for letter in text:
        symbol1 = caesar(letter, shift3)
        symbol2 = rotor(symbol1, rot3, reverse=False)
        symbol3 = caesar(symbol2, shift2 - shift3)
        symbol4 = rotor(symbol3, rot2, reverse=False)
        symbol5 = caesar(symbol4, shift1 - shift2)
        symbol6 = rotor(symbol5, rot1, reverse=False)
        symbol7 = caesar(symbol6, -shift1)
        symbol8 = reflector(symbol7, ref)
        symbol9 = caesar(symbol8, shift1)
        symbol10 = rotor(symbol9, rot1, reverse=True)
        symbol11 = caesar(symbol10, shift2 - shift1)
        symbol12 = rotor(symbol11, rot2, reverse=True)
        symbol13 = caesar(symbol12, shift3 - shift2)
        symbol14 = rotor(symbol13, rot3, reverse=True)
        symbol15 = caesar(symbol14, -shift3)
        encrypted += symbol15
    return encrypted

But that's not all: the rotors of a real Enigma turn right during the operation of the machine.
After pressing a key on the keyboard and before transferring it to the right rotor, it is shifted by 1 position each time any button is pressed. That is, if the position of the rotor was 0 at the beginning of the encryption process, then the first character is encrypted with an offset of +1 on the right rotor. The next character is encrypted with an offset +2. etc.
Each rotor has a special mechanism that allows you to turn the neighboring one (located to the left) the rotor under certain conditions. The condition is to achieve a certain offset:

    Rotor 1-at 17
    Rotor 2-at 5
    Rotor 3-at 22
    
Now let's impove our enigma function and take new features into account.

In [6]:
def enigma(text, ref, rot1, shift1, rot2, shift2, rot3, shift3):
    dictionary = {3: 22, 2: 5, 1: 17}
    neighbor = False
    encrypted = ''
    text = [i for i in text if i.isalpha()]
    text = ''.join(text).upper()
    for letter in text:
        shift3 += 1
        if shift3 == dictionary.get(rot3):
            shift2 += 1
        if neighbor:
            shift2 += 1
            shift1 += 1
            neighbor = False
        if shift2 == dictionary.get(rot2) - 1:
            neighbor = True
        if shift3 == 26:
            shift3 = 0
        if shift2 == 26:
            shift2 = 0
        if shift1 == 26:
            shift1 = 0
        symbol1 = caesar(letter, shift3)
        symbol2 = rotor(symbol1, rot3, reverse=False)
        symbol3 = caesar(symbol2, shift2 - shift3)
        symbol4 = rotor(symbol3, rot2, reverse=False)
        symbol5 = caesar(symbol4, shift1 - shift2)
        symbol6 = rotor(symbol5, rot1, reverse=False)
        symbol7 = caesar(symbol6, -shift1)
        symbol8 = reflector(symbol7, ref)
        symbol9 = caesar(symbol8, shift1)
        symbol10 = rotor(symbol9, rot1, reverse=True)
        symbol11 = caesar(symbol10, shift2 - shift1)
        symbol12 = rotor(symbol11, rot2, reverse=True)
        symbol13 = caesar(symbol12, shift3 - shift2)
        symbol14 = rotor(symbol13, rot3, reverse=True)
        symbol15 = caesar(symbol14, -shift3)
        encrypted += symbol15.upper()
    return encrypted

Now pay attention to the pairwise connections using flexible wires at the bottom. These are the connectors corresponding to all the symbols.

Connecting them in pairs with wires, the operator replaces them (in each such pair) before sending them for encryption and after.

If some characters are not switched, then they are not replaced.


Imagine we use Rotors 1, 2, 3 and a reflector of type "B", without an initial offset.

Usually, with such encryption, we get "B" from the symbol "A".

Commute 2 characters: "A" and "With".

Now, BEFORE encryption, "A" will turn into"C". "C" is encrypted in "Q".

If we commuted 2 pairs of characters: "A" and " C " + "Q" and "D", the process would look like this:

    BEFORE encryption, "A" will turn into"C".
    "C" is encrypted in "Q".
    AFTER encryption, "Q" turns into "D".

# Enigma v.1

Now let's add pairs argument to our enigma function. This argument will be used for string character replacement.

In [7]:
def enigma(text, ref, rot1, shift1, rot2, shift2, rot3, shift3, pairs=""):
    pairs = pairs.replace(' ', '').upper()
    if len(pairs) != len(set(pairs)):
        return 'Sorry, it is impossible to perform switching'
    if pairs:
        pairs = [pairs[i:i+2] for i in range(0, len(pairs), 2)]
        dict_pairs = {}
        for pair in pairs:
            pair = pair.upper()
            dict_pairs[pair[0]] = pair[1]
            dict_pairs[pair[1]] = pair[0]
    dictionary = {3: 22, 2: 5, 1: 17}
    neighbor = False
    encrypted = ''
    text = [i for i in text if i.isalpha()]
    text = ''.join(text).upper()
    for letter in text:
        if pairs:
            if letter in dict_pairs.keys():
                letter = dict_pairs.get(letter)
        shift3 += 1
        if shift3 == dictionary.get(rot3):
            shift2 += 1
        if neighbor:
            shift2 += 1
            shift1 += 1
            neighbor = False
        if shift2 == dictionary.get(rot2) - 1:
            neighbor = True
        if shift3 == 26:
            shift3 = 0
        if shift2 == 26:
            shift2 = 0
        if shift1 == 26:
            shift1 = 0
        symbol1 = caesar(letter, shift3)
        symbol2 = rotor(symbol1, rot3, reverse=False)
        symbol3 = caesar(symbol2, shift2 - shift3)
        symbol4 = rotor(symbol3, rot2, reverse=False)
        symbol5 = caesar(symbol4, shift1 - shift2)
        symbol6 = rotor(symbol5, rot1, reverse=False)
        symbol7 = caesar(symbol6, -shift1)
        symbol8 = reflector(symbol7, ref)
        symbol9 = caesar(symbol8, shift1)
        symbol10 = rotor(symbol9, rot1, reverse=True)
        symbol11 = caesar(symbol10, shift2 - shift1)
        symbol12 = rotor(symbol11, rot2, reverse=True)
        symbol13 = caesar(symbol12, shift3 - shift2)
        symbol14 = rotor(symbol13, rot3, reverse=True)
        symbol15 = caesar(symbol14, -shift3)
        if pairs:
            if symbol15 in dict_pairs.keys():
                symbol15 = dict_pairs.get(symbol15)
        encrypted += symbol15
    return encrypted

Now let's encrypt some text

In [8]:
enigma('The target was found', 1, 1, 0, 2, 0, 3, 0)

'OPCFOTZYHJKAKCTEA'